# Classical Ciphers Workshop

This workshop is presented as a Jupyter notebook, which is basically a list of cells. For example, this is a Markdown cell which contains text. In this notebook you will also find code cells, which contain Python code that you can run and edit.

To run a cell, click on it and hit `Shift+Enter`. Try running the code cell below.

In [ ]:
a = "banana"
b = 3
c = 4
2 * b + c

Note that a code cell will output the last expression evaluated (so you don't need to explicitly call `print`).

To edit a code cell, you can click into it, or if you've selected the cell you can hit `Enter`. Try changing some of the variables around in the first code cell and re-running it.

Some tips on using Jupyter notebooks effectively:
- You should generally be running cells in the order that they're presented in, and you shouldn't be skipping over any cells.
- You can go back to run previous cells, but you should be careful that this doesn't cause any unintended effects. For example, re-running the cell below will modify `a` repeatedly, which may not be what you expect. A good rule of thumb is that you shouldn't be overwriting variables that were defined in previous cells.
- If something seems messed up, try running all the cells again starting from the top of the notebook. You can basically just spam `Shift+Enter` to accomplish this.

In [ ]:
a = a + "na"
a

## Background: Caesar Cipher

In this section we'll be doing some work with the Caesar cipher, including frequency analysis. You won't be implementing anything here, but you should know how all of these functions work so that you can use them to attack the Vigenere Cipher later.

Read the code in the cells below, and run them to see their output.

In [ ]:
import string

alphabet = string.ascii_lowercase
alphabet

In [ ]:
def rotate_char(c: str, shift: int) -> str:
    # Caesar shift applied to a single character
    if c in alphabet:
        # Apply Caesar shift
        index = alphabet.index(c)
        new_index = (index + shift) % len(alphabet)
        return alphabet[new_index]
    else:
        # Return c unchanged
        return c

def rotate(text: str, shift: int) -> str:
    # Apply Caesar shift to a multi-character string
    output = ""
    for character in text:
        output += rotate_char(character, shift)
    return output

In [ ]:
ciphertext = rotate("attack at dawn", 3)
ciphertext

Of course, the major weakness of the Caesar cipher is that there's only 26 possible keys. Here, we loop over all 26 possible keys and print a decryption under each one.

In [ ]:
for key in range(26):
    print(key, rotate(ciphertext, -key))

Now we have 26 possible plaintexts, and it's not hard to pick out the correct one. It's a bit tedious to read all 26 decryptions though, and ideally we'd like to automate the process of finding the "most English-like" text. To do this, we'll use a tool called *frequency analysis*. In short, we expect certain letters like `e` and `t` to be more common in English text than others, like `j` or `q`. The code cell below will define our reference for how common each English letter is. For instance, in any given sample of English text, we expect about 12.7% of the letters to be `e`s.

In [ ]:
LETTER_FREQUENCIES = {"a": 0.08167, "b": 0.01492, "c": 0.02782, "d": 0.04253, "e": 0.12702, "f": 0.02228, "g": 0.02015, "h": 0.06094, "i": 0.06966, "j": 0.00153, "k": 0.00772, "l": 0.04025, "m": 0.02406, "n": 0.06749, "o": 0.07507, "p": 0.01929, "q": 0.00095, "r": 0.05987, "s": 0.06327, "t": 0.09056, "u": 0.02758, "v": 0.00978, "w": 0.02360, "x": 0.00150, "y": 0.01974, "z": 0.00075}

Next, we'll define a function to count the occurrences of each letter in a given text. The [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter) object makes this pretty easy to implement.

In [ ]:
from collections import Counter
def get_letter_counts(text: str) -> Counter:
    letter_counts = Counter()
    letter_counts.update([char for char in text if char in alphabet])
    return letter_counts

In [ ]:
get_letter_counts("attack at dawn")

The function below will give us a "distance" between the letter counts of a text and the expected counts based on typical English frequencies. You don't need to know how the details of this function works, although if you've taken a statistics class you may recognize the [$\chi^2$ test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test#Calculating_the_test-statistic) that's being used here. In general, the lower the result, the "closer" to English that the text is.

In [ ]:
def distance_from_english(text: str) -> float:
    from scipy.stats import chisquare
    counts = get_letter_counts(text)
    total = counts.total()
    f_observed = [counts[letter] for letter in alphabet]
    f_expected = [LETTER_FREQUENCIES[letter] * total for letter in alphabet]
    chi2_statistic, p_value = chisquare(f_obs=f_observed, f_exp=f_expected)
    return float(chi2_statistic)

Check out the outputs below to get a feel for the kinds of numbers this function should be returning.

In [ ]:
print(f'{distance_from_english("attack at dawn") = }') # This is English, so we should get something pretty low!
print(f'{distance_from_english("tmmtvd tm wtpg") = }') # This is also somewhat low since the letters here are pretty common.
print(f'{distance_from_english("exxego ex hear") = }') # Despite e being the most common letter, the presence of multiple `x`s increases the distance significantly.
print(f'{distance_from_english("qjjqsa qj tqmd") = }') # This distance is huge since it contains a lot of uncommon letters, and not enough common letters.

Let's see how well this works on another ciphertext:

In [ ]:
ciphertext = "pm ufodvwbu hvs tfseisbqwsg ct zshhsfg wb hvs qwdvsfhslh, obr pm ybckwbu hvs sldsqhsr rwghfwpihwcb ct hvcgs zshhsfg wb hvs cfwuwboz zobuious ct hvs dzowbhslh, o viaob qob sogwzm gdch hvs jozis ct hvs gvwth pm zccywbu oh hvs rwgdzoqsasbh ct dofhwqizof tsohifsg ct hvs ufodv. hvwg wg ybckb og tfseisbqm obozmgwg"
for key in range(26):
    decrypted = rotate(ciphertext, -key)
    distance = distance_from_english(decrypted)
    print(f"{key:2d}: {distance = :7.2f}, {decrypted[:50]}...")

As expected, the correct decryption has the lowest distance.

Now we'll put it all together to give us the "best" decryption for any ciphertext.

In [ ]:
def get_best_decryption(ciphertext: str) -> str:
    candidates = []
    for key in range(26):
        decrypted = rotate(ciphertext, -key)
        distance = distance_from_english(decrypted)
        candidates.append((distance, decrypted))
    best_distance, best_decrypted = min(candidates)
    return best_decrypted

In [ ]:
get_best_decryption(ciphertext)

## Workshop: Vigenere Cipher

Here's the ciphertext that we'll be trying to decrypt.

In [ ]:
our_ciphertext = "Qo eg vlbekakcdah, ffbunr irpujx, B kpo ps uv yi erofjlah tj neowbsju er myc bezxj mv ic lzv sdmeupuj. M xeaeqvtxc jdif km zkmg kcqi wifpjo. M tkrujh mycyn xxvl fwvmzci smmy rxaq mf cdoykv le zvbeiyjk hi ybysafj yo sg kfu lvxdgiaw. B bcul e yrrxavep coa sg kfu YHl kfuu pbjruj xh rlt plx jfesw myco semtf, jdi vfkfwrr kfuu oxvn qjh myc rksdj rxac kvyt. Usn tmkhh lrw Y'i e ffbuh tticdp. Qr tfyhhkvl xwzx eclav yrgbah mf kqgi fv nhkyw, rlt E gte qqu abkfeqx myc ihmzyruox xdzuhpbjfcarm kfqp M artu plx wgdawm wycepr zl jdi NJY. Jss rvyho ezf, ko smyv Aqnse rlt E hxtgtah myyj kyk tfyhhkvl'i ahntyjesg nmkhh gfr ra ghdnbaxx ngjdsnk qeii zimkjhbee yj qhuchj ghdnkpikj. Re plbj cdz, ax smkclm fsh ylbcbhar t spqjh gvu Skqiro jk pxrpd smmy. Rxa obuq xwh t cmj kj yll komgx rxa ltebvqp hw yflpbtyjesg gpecvtdq ma'h ufswdx, llax ww Tumra'w Iymjkwafn qjh Fzahkwhwr'i Ssku, ydz qr ngva egu G mavx gjuwwxu rxwx hlp wejm nyi nivvglah lf uuhp. Hlp ikr Ivrun atj keox xerhwrvvb ru xav burmvv, ydz fxtyca unzru w tkf yj oykwgdc xav lup. Aavl Faxxi zuceg km iligu uxkpx uyoo sg kfu ievygda, M uvaqii vflsavgvb, rqx Vrpeh ewmgiah fv re yeed besr, teb jdem zr mww hejo w ttjqyjk iyyia. M prq skrmvlj ps ufu jk lxi cnlikzcdyi tj y ckxavp, kjxbc mkn chllwawm uykclmvp, Serwp, axwvzvb yjxh kfu hmozlw nshd mda rbxfj ps ucshp snk: \"Nupik zq q ysfgsjav araaav!\""

### Some helper functions

The `extract_letters` function will take in a string, and return another string with only the letters of the original string (converted to lowercase). You can think of this function as removing spaces, punctuation, and capitalization.

The `replace_letters` function takes in an original string and a string of letters, and returns the orignal string but with its letters replaced. You can think of this function as restoring spaces, punctuation, and capitalization after we changed all the letters.

See the examples after this code block to see how to use these functions.

In [ ]:
def extract_letters(text: str) -> str:
    return ''.join(i.lower() for i in text if i.lower() in alphabet)

def replace_letters(original_text: str, new_letters: str) -> str:
    new_text = ''
    new_letters_iter = iter(new_letters)
    for character in original_text:
        if character in alphabet:
            new_text += next(new_letters_iter)
        elif character.lower() in alphabet:
            new_text += next(new_letters_iter).upper()
        else:
            new_text += character
    return new_text

In [ ]:
sentence = "The quick brown fox jumps over the lazy dog."
letters = extract_letters(sentence)
letters

In [ ]:
new_letters = "ourquickbrownfoxjumpsoverthelazycat"
replaced = replace_letters(sentence, new_letters)
replaced

The `interleave` function below takes in a list of strings and returns the interleaving of the characters from the input strings. See the example below on how to use it.

In [ ]:
def interleave(texts: list[str]) -> str:
    total_length = len(''.join(texts))
    result = [''] * total_length
    for i in range(len(texts)):
        result[i::len(texts)] = texts[i]
    return ''.join(result)

In [ ]:
interleave(["0369cf", "147ad", "258be"])

The `uninterleave` function does the opposite of `interleave`, creating `n` texts such that the `k`th text has every `n`th character starting from index `k`. Sorry I don't have a better name for this function.

In [ ]:
def uninterleave(text: str, n: int) -> list[str]:
    streams = []
    for i in range(n):
        streams.append(text[i::n])
    return streams

In [ ]:
uninterleave("0123456789abcdef", 3)

**YOUR TASK**: Implement Vigenere encryption of a text using a key. Some code is already written for you.

A basic outline of the process:
- We can implement the Vigenere cipher as multiple Caesar ciphers, one for each letter in the key.
- Use the `uninterleave` function to get the `key_len` separate streams to be affected by each Caesar cipher.
- Use the `rotate` function to apply a Caesar shift to each stream. You should shift the `i`th stream by a shift value of `shifts[i]`.
- When you're done, `interleave` the streams back together.

In [ ]:
def vigenere_encrypt(original_text: str, key: str) -> str:
    key_len = len(key)
    shifts = [alphabet.index(key_char) for key_char in key.lower()]
    letters = extract_letters(original_text)

    # YOUR TASK: Implement the Vigenere cipher on `letters`, and store the result into `new_letters`.
    new_letters = ...

    return replace_letters(original_text, new_letters)

In [ ]:
plaintext = "If the recipient of the message knows the key, they can recover the plaintext by reversing this process."
ciphertext = "Dn glk rvxqcmknk jn glk mvnankk kejef xne bzg, glky tvv eiiomzz glk pcvqaxkxk wg eibeinqak zhzn xesiejn."
assert vigenere_encrypt(plaintext, "vinegar") == ciphertext, f"Incorrect ciphertext: {vigenere_encrypt(plaintext, 'vinegar')}"

If the above code ran without error, then you've implemented Vigenere encryption correctly. Hooray!

**YOUR TASK**: Implement the `vigenere_decrypt` function below. You can more or less copy/paste your `vigenere_encrypt` implementation here, but we should be doing a Caesar shift in the other direction to undo the encryption step.

In [ ]:
def vigenere_decrypt(original_text: str, key: str) -> str:
    # YOUR TASK: Implement Vigenere decryption.
    ...

    return replace_letters(original_text, new_letters)

The code below should now run without error.

In [ ]:
assert vigenere_decrypt(ciphertext, "vinegar") == plaintext

### Determining the most likely key length

Our first step in breaking the Vigenere cipher will be to find the length of the key. To do this, we're going to use a metric called the *index of coincidence*, or IOC. You can think of this as a number that summarizes the frequency of the characters in the text. Formally, the IOC is defined as the probability that two randomly selected characters from the text are the same, multiplied by some normalizing factor. In our case, the normalizing factor is 26.

To get a feel for what the IOC represents, take a look at the examples below, where we compute an IOC for various kinds of text.

In [ ]:
import random
random.seed(1) # This is here so we can get reproducible results.

# This approximately computes the IOC by randomly sampling letters from the text. You'll write an exact implementation later.
def ioc_sample(letters: str, n_samples: int = 100000) -> float:
    n_same = 0
    for _ in range(n_samples):
        x, y = random.sample(letters, 2)
        if x == y:
            n_same += 1
    return n_same / n_samples * 26

random_letters = ''.join(random.choices(alphabet, k=1000)) # 1000 random letters
only_As = 'a' * 1000
a_few_letters = 'aaaaaaaaaaaaaaaaaaaaaabbbbbbbbccccdefg'
english_text = extract_letters("The index of coincidence provides a measure of how likely it is to draw two matching letters by randomly selecting two letters from a given text.")

print(f"{ioc_sample(random_letters) = }")
print(f"{ioc_sample(a_few_letters) = }")
print(f"{ioc_sample(only_As) = }")
print(f"{ioc_sample(english_text) = }")

These results should make some sense to you. If we have random letters with each letter being equally likely, the probability that the second letter we sample is the same as the first is $\frac{1}{26}$. When we multiply by the normalization factor of 26, this becomes 1. So the IOC of random letters should be around 1.

If there's a letter that's more common, then our IOC should increase, since it's more likely that we'll pick two of that letter. In the extreme case, if there's only one letter than we'll always pick the same letter twice with probability 1, which is normalized to 26. In general, the IOC should range between 1 (random text) and 26 (only one letter).

Now let's implement an exact calculation of the IOC. Note that the IOC only depends on the *frequency* of the letters: if we shuffle the letters around, the probability of two random letters being the same shouldn't change.

How do we get the probability that two randomly selected letters are the same? Well, if they are the same, then one of the following is true:
- The first letter is `a`, and the second letter is also `a`
- The first letter is `b`, and the second letter is also `b`
- ...
- The first letter is `z`, and the second letter is also `z`

Let's say our text has $N$ letters, and suppose `a` appears $k$ times. Then probability that the first letter is `a` is $\frac{k}{N}$. When we choose the second letter, there are now $k-1$ `a`s and $N-1$ letters, so the probability that the second letter is `a` is $\frac{k-1}{N-1}$. So the probability that both letters are `a` is $\frac{k(k-1)}{N(N-1)}$. (Convince yourself that this is true even if $k$ is 0 or 1.)

We repeat this computation for each letter, and in the end we can just add the probabilities together. Finally, we multiply by the normalization factor of 26.

**YOUR TASK**: Fill in the code to implement an exact calculation for the IOC.

In [ ]:
def ioc(text: str) -> float:
    letter_counts = get_letter_counts(text)
    n_letters = letter_counts.total()
    probability = 0.0
    for letter in alphabet:
        # YOUR TASK: add the probability that two randomly selected letters are both `letter`
        probability += ...

    return probability * 26

When you've finished your implementation, run the code below and make sure the outputs are similar to what we got above.

In [ ]:
print(f"{ioc(random_letters) = }")
print(f"{ioc(a_few_letters) = }")
print(f"{ioc(only_As) = }")
print(f"{ioc(english_text) = }")

Since the IOC depends only on frequencies, and we've defined frequencies for the English language, we also can compute a theoretical IOC for English text:

In [ ]:
IOC_ENGLISH = 26 * sum(freq**2 for freq in LETTER_FREQUENCIES.values())
IOC_ENGLISH

Now, why do we care about the IOC? Well, it has a few key properties that make it useful for us:
- As mentioned before, the IOC depends only on the frequency of letters, and not their ordering. So even if we shuffle the letters around, the IOC is still the same.
- The IOC doesn't care about what letters have what frequency. So if we swapped all the `a`s and `b`s in a text, the IOC is still the same. In particular, the IOC remains the same *even if we apply a Caesar shift to the text*. (see code below)
- Finally, the IOC allows us to distinguish between "random" text and English text that's been run through a Caesar shift. Namely, random text will have an IOC around 1, and English text will have an IOC around 1.7 (see above).

In [ ]:
assert ioc(rotate(english_text, 5)) == ioc(english_text)

Now that we know what the IOC does, let's use it to find the most likely key length for a given ciphertext. Our procedure for this will be as follows:
- Split the ciphertext into streams based on some key length $k$
- For each stream, compute the IOC
- Take the average IOC across all the streams

Let's say we have a large ciphertext, which was encrypted using the Vigenere cipher with a key length of 6. Some things to think about:
- If we guess the correct key length $k=6$, what do we expect the average IOC to be?
- What if we guess a key of length $k=12$?
- What if we guess a key of length $k=5$?

**YOUR TASK**: Implement the above procedure to get the average IOC for a given key length. Here's a rough outline of the steps:
- Use `uninterleave` to get the streams for the ciphertext
- You can use the `ioc` function above to get the IOC of each stream

In [ ]:
def average_ioc_for_key_length(text: str, key_len: int) -> float:
    letters = extract_letters(text)
    total_ioc = 0.0

    # YOUR TASK: Compute the average IOC of letters for a key length of `key_len`
    ...

    return total_ioc / key_len

Let's take our ciphertext and calculate the average IOC for each key length. The results are shown in the bar graph below, with the red dashed line being the IOC of English text.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

key_lens = list(range(1, 31))
avg_iocs = []
for key_len in key_lens:
    avg_iocs.append(average_ioc_for_key_length(our_ciphertext, key_len))

fig, ax = plt.subplots()
fig.set_tight_layout(True)
ax.barh(key_lens, avg_iocs)
ax.set_yticks(key_lens, labels=key_lens)
ax.set_xlabel("Average IOC")
ax.set_ylabel("Key length")
ax.vlines(IOC_ENGLISH, 0, max(key_lens)+1, color='r', linestyle='dashed')
ax.invert_yaxis()

Based on the output, what do you think is the most likely key length? Do these results make sense?

### Decrypting the message

**YOUR TASK**: Implement `get_best_vigenere_decryption`. Some hints:
- We pass in the key length as a parameter. If you split the text into `key_len` streams, each stream acts as a Caesar cipher.
- We know how to decrypt Caesar ciphers! (Hint: `get_best_decryption`)
- When you're done decrypting each stream, recombine them into a single text.

In [ ]:
def get_best_vigenere_decryption(text: str, key_len: int) -> str:
    letters = extract_letters(text)

    # YOUR TASK: Store the best decryption of `letters` into `new_letters`
    new_letters = ...

    return replace_letters(text, new_letters)

Test your function by filling in the key length you got from the previous section, and running the command below.

In [ ]:
key_len = ... # YOUR TASK: fill in the key length from the previous section
print(get_best_vigenere_decryption(our_ciphertext, key_len))

And that's it!

Some more things to think about/implement if you have time:
- Try recovering the key used to encrypt the message
- Why does `get_best_decryption` work in our implementation of `get_best_vigenere_decryption`?
- In `distance_from_english`, we return `float(chi2_statistic)` instead of `float(p_value)`.
    - You might be tempted to think of `p_value` as a probability that the text is in English. What is the problem with this thinking? (Hint: try changing the implementation to return `float(p_value)` and test it on some examples. Are any of the outputs surprising?)
    - What does `p_value` really represent?